# `pandas` cheatsheet

In [1]:
import numpy as np
import pandas as pd

## General functions

### 1. Data manipulations

* pandas.melt
* pandas.pivot, pandas.pivot_table
* pandas.crosstab
* pandas.cut, pandas.qcut
* pandas.merge, pandas.merge_ordered, pandas.merge_asof, pandas.concat
* pandas.get_dummies
* pandas.factorize
* pandas.unique
* pandas.wide_to_long

1. `pandas.melt`: Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.

In [2]:
# pd.melt?

In [3]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [4]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [5]:
pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [6]:
pd.melt(df, id_vars=['A'], value_vars=['B'],
        var_name='myVarname', value_name='myValname')

,A,myVarname,myValname
0,a,B,1
1,b,B,3
2,c,B,5


In [7]:
df.columns = [list('ABC'), list('DEF')]
df

,A,B,C
,D,E,F
0,a,1,2
1,b,3,4
2,c,5,6


In [8]:
pd.melt(df, col_level=0, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [9]:
pd.melt(df, id_vars=[('A', 'D')], value_vars=[('B', 'E')])

,"(A, D)",variable_0,variable_1,value
0,a,B,E,1
1,b,B,E,3
2,c,B,E,5


2. `pandas.pivot` Return reshaped DataFrame organized by given index / column values.

In [10]:
# pd.pivot?

In [11]:
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two',
                           'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


In [12]:
df.pivot(index='foo', columns='bar')
# df.pivot(index='foo', columns='bar', values=['baz', 'zoo'])

baz       zoo      
bar   A  B  C   A  B  C
foo                    
one   1  2  3   x  y  z
two   4  5  6   q  w  t

In [13]:
df.pivot(index='foo', columns='bar')['baz']
# df.pivot(index='foo', columns='bar', values='baz')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


In [14]:
# # A ValueError is raised if there are any duplicates.
# df = pd.DataFrame({"foo": ['one', 'one', 'two', 'two'],
#                    "bar": ['A', 'A', 'B', 'C'],
#                    "baz": [1, 2, 3, 4]})
# df.pivot(index='foo', columns='bar', values='baz')

`pandas.pivot_table` Create a spreadsheet-style pivot table as a DataFrame.

In [15]:
# pd.pivot_table?

In [16]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [17]:
# Aggregate values by taking the sum
# missing values using the fill_value parameter
pd.pivot_table(df, values='D', index=['A', 'B'], 
               columns=['C'], aggfunc=np.sum, fill_value=0)

C        large  small
A   B                
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6

In [18]:
# Aggregates by taking the mean/sum across multiple columns
pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean, 'E': np.sum})

D   E
A   C                  
bar large  5.500000  15
    small  5.500000  17
foo large  2.000000   9
    small  2.333333  13

In [19]:
pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean, 'E': [min, max, np.mean]})

D    E               
               mean  max      mean  min
A   C                                  
bar large  5.500000  9.0  7.500000  6.0
    small  5.500000  9.0  8.500000  8.0
foo large  2.000000  5.0  4.500000  4.0
    small  2.333333  6.0  4.333333  2.0

3. `pandas.crosstab` Compute a simple cross tabulation of two (or more) factors. By default computes a frequency table of the factors unless an array of values and an aggregation function are passed.

In [20]:
# pd.crosstab?

In [21]:
a = np.array(["foo", "foo", "foo", "foo", "bar", "bar",
              "bar", "bar", "foo", "foo", "foo"], dtype=object)
b = np.array(["one", "one", "one", "two", "one", "one",
              "one", "two", "two", "two", "one"], dtype=object)
c = np.array(["dull", "dull", "shiny", "dull", "dull", "shiny",
              "shiny", "dull", "shiny", "shiny", "shiny"],
             dtype=object)
pd.crosstab(a, [b, c], rownames=['a'], colnames=['b', 'c'])

b    one        two      
c   dull shiny dull shiny
a                        
bar    1     2    1     0
foo    2     2    1     2

In [22]:
foo = pd.Categorical(['a', 'b'], categories=['a', 'b', 'c'])
bar = pd.Categorical(['d', 'e'], categories=['d', 'e', 'f'])
pd.crosstab(foo, bar)

col_0,d,e
row_0,,
a,1,0
b,0,1


In [23]:
pd.crosstab(foo, bar, dropna=False)

col_0,d,e,f
row_0,,,
a,1,0,0
b,0,1,0
c,0,0,0


4. `pandas.cut`: Bin values into discrete intervals.

In [24]:
# Discretize into three equal-sized bins.
pd.cut(np.array([1, 7, 5, 4, 6, 3]), 3, retbins=True)

([(0.994, 3.0], (5.0, 7.0], (3.0, 5.0], (3.0, 5.0], (5.0, 7.0], (0.994, 3.0]]
 Categories (3, interval[float64]): [(0.994, 3.0] < (3.0, 5.0] < (5.0, 7.0]],
 array([0.994, 3.   , 5.   , 7.   ]))

In [25]:
# Discovers the same bins, but assign them specific labels. Notice that the returned Categorical’s categories are labels and is ordered.
pd.cut(np.array([1, 7, 5, 4, 6, 3]), 3, labels=["bad", "medium", "good"])

[bad, good, medium, medium, good, bad]
Categories (3, object): [bad < medium < good]

In [26]:
# labels=False implies you just want the bins back.
pd.cut([0, 1, 1, 2], bins=4, labels=False)

array([0, 1, 1, 3])

In [27]:
# Passing a Series as an input returns a Series with categorical dtype:
s = pd.Series(np.array([2, 4, 6, 8, 10]), index=['a', 'b', 'c', 'd', 'e'])
pd.cut(s, 3)

a    (1.992, 4.667]
b    (1.992, 4.667]
c    (4.667, 7.333]
d     (7.333, 10.0]
e     (7.333, 10.0]
dtype: category
Categories (3, interval[float64]): [(1.992, 4.667] < (4.667, 7.333] < (7.333, 10.0]]

In [28]:
# Passing a Series as an input returns a Series with mapping value. It is used to map numerically to intervals based on bins.
s = pd.Series(np.array([2, 4, 6, 8, 10]),
              index=['a', 'b', 'c', 'd', 'e'])
pd.cut(s, [0, 2, 4, 6, 8, 10], labels=False, retbins=True, right=True)

(a    0
 b    1
 c    2
 d    3
 e    4
 dtype: int64, array([ 0,  2,  4,  6,  8, 10]))

In [29]:
# Use drop optional when bins is not unique
pd.cut(s, [0, 2, 4, 6, 10, 10], labels=False, retbins=True,
       right=True, duplicates='drop')

(a    0
 b    1
 c    2
 d    3
 e    3
 dtype: int64, array([ 0,  2,  4,  6, 10]))

In [30]:
# Passing an IntervalIndex for bins results in those categories exactly. 
# Notice that values not covered by the IntervalIndex are set to NaN. 
# 0 is to the left of the first bin (which is closed on the right), and 1.5 falls between two bins.
bins = pd.IntervalIndex.from_tuples([(0, 1), (2, 3), (4, 5)])
pd.cut([0, 0.5, 1.5, 2.5, 4.5], bins)

[NaN, (0.0, 1.0], NaN, (2.0, 3.0], (4.0, 5.0]]
Categories (3, interval[int64]): [(0, 1] < (2, 3] < (4, 5]]

   `pandas.qcut`: Quantile-based discretization function.

In [31]:
pd.qcut(range(5), 4)
pd.qcut(range(5), 3, labels=["good", "medium", "bad"])
pd.qcut(range(5), 4, labels=False)

array([0, 0, 1, 2, 3])

5. `pandas.merge` Merge DataFrame or named Series objects with a database-style join.

In [32]:
# pd.merge?

In [33]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})
df1

,lkey,value
0,foo,1
1,bar,2
2,baz,3
3,foo,5


In [34]:
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]})
df2

,rkey,value
0,foo,5
1,bar,6
2,baz,7
3,foo,8


In [35]:
# Merge df1 and df2 on the lkey and rkey columns. 
# The value columns have the default suffixes, _x and _y, appended.
df1.merge(df2, left_on='lkey', right_on='rkey')

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


In [36]:
# Merge DataFrames df1 and df2 with specified left and right suffixes appended to any overlapping columns.
df1.merge(df2, left_on='lkey', right_on='rkey',
          suffixes=('_left', '_right'))

,lkey,value_left,rkey,value_right
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


`pandas.merge_ordered`Perform merge with optional filling/interpolation.

`pandas.merge_asof` Perform an asof merge. This is similar to a left-join except that we match on nearest key rather than equal keys

In [37]:
left = pd.DataFrame({'a': [1, 5, 10], 'left_val': ['a', 'b', 'c']})
left

,a,left_val
0,1,a
1,5,b
2,10,c


In [38]:
right = pd.DataFrame({'a': [1, 2, 3, 6, 7],
                      'right_val': [1, 2, 3, 6, 7]})
right

,a,right_val
0,1,1
1,2,2
2,3,3
3,6,6
4,7,7


In [39]:
pd.merge_asof(left, right, on='a')

,a,left_val,right_val
0,1,a,1
1,5,b,3
2,10,c,7


`pandas.concat`  Concatenate pandas objects along a particular axis with optional set logic along the other axes; Can also add a layer of hierarchical indexing on the concatenation axis, which may be useful if the labels are the same (or overlapping) on the passed axis number.

In [40]:
# pd.concat?

In [41]:
# Combine two Series
s1 = pd.Series(['a', 'c'])
s2 = pd.Series(['b', 'd'])
pd.concat([s1, s2])

0    a
1    c
0    b
1    d
dtype: object

In [42]:
# Clear the existing index and reset it in the result by setting the ignor_index option to True
pd.concat([s1, s2], ignore_index=True)

0    a
1    c
2    b
3    d
dtype: object

In [43]:
# Add a hierarchical index at the outermost level of the data with the keys option.
# Label the index keys you create with the names option.
pd.concat([s1, s2], keys=['s1', 's2'])
pd.concat([s1, s2], keys=['s1', 's2'], names=['Series name', 'Row ID'])

Series name  Row ID
s1           0         a
             1         c
s2           0         b
             1         d
dtype: object

In [44]:
# Combine two DataFrame objects with identical columns.
df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 3], ['d', 4]],
                   columns=['letter', 'number'])
pd.concat([df1, df2])

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [45]:
# Combine DataFrame objects with overlapping columns and return everything. Columns outside the intersection will be filled with NaN values.
df3 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']],
                   columns=['letter', 'number', 'animal'])
df3

,letter,number,animal
0,c,3,cat
1,d,4,dog


In [46]:
pd.concat([df1, df3], sort=False)

,letter,number,animal
0,a,1,NaN
1,b,2,NaN
0,c,3,cat
1,d,4,dog


In [47]:
# Combine DataFrame objects with overlapping columns and return only those that are shared by passing inner to the join keyword argument.
pd.concat([df1, df3], join="inner")

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [48]:
# Combine DataFrame objects horizontally along the x axis by passing in axis=1.
df4 = pd.DataFrame([['bird', 'polly'], ['monkey', 'george']],
                   columns=['animal', 'name'])
pd.concat([df1, df4], axis=1)

,letter,number,animal,name
0,a,1,bird,polly
1,b,2,monkey,george


6. `pandas.get_dummies` Convert categorical variable into dummy/indicator variables.

In [49]:
# pd.get_dummies

In [50]:
s = pd.Series(list('abca'))
pd.get_dummies(s)

,a,b,c
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0


In [51]:
type(pd.get_dummies(s))

pandas.core.frame.DataFrame

In [52]:
pd.get_dummies(pd.Series(list('abcaa')))

,a,b,c
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0
4,1,0,0


In [53]:
pd.get_dummies(pd.Series(list('abcaa')), drop_first=True)

,b,c
0,0,0
1,1,0
2,0,1
3,0,0
4,0,0


In [54]:
pd.get_dummies(pd.Series(list('abc')), dtype=float)

,a,b,c
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0


In [55]:
s1 = ['a', 'b', np.nan]
pd.get_dummies(s1)

,a,b
0,1,0
1,0,1
2,0,0


In [56]:
pd.get_dummies(s1, dummy_na=True)

,a,b,NaN
0,1,0,0
1,0,1,0
2,0,0,1


In [57]:
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['b', 'a', 'c'],
                   'C': [1, 2, 3]})
df

,A,B,C
0,a,b,1
1,b,a,2
2,a,c,3


In [58]:
pd.get_dummies(df, prefix=['col1', 'col2'])

,C,col1_a,col1_b,col2_a,col2_b,col2_c
0,1,1,0,0,1,0
1,2,0,1,1,0,0
2,3,1,0,0,0,1


7. `pandas.factorize` Encode the object as an enumerated type or categorical variable.

In [59]:
codes, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b'])
codes, uniques

(array([0, 0, 1, 2, 0]), array(['b', 'a', 'c'], dtype=object))

In [60]:
# With sort=True, the uniques will be sorted, and codes will be shuffled so that the relationship is the maintained.
codes, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b'], sort=True)
codes, uniques

(array([1, 1, 0, 2, 1]), array(['a', 'b', 'c'], dtype=object))

In [61]:
# Missing values are indicated in codes with na_sentinel (-1 by default). 
# Note that missing values are never included in uniques.
codes, uniques = pd.factorize(['b', None, 'a', 'c', 'b'])
codes, uniques

(array([ 0, -1,  1,  2,  0]), array(['b', 'a', 'c'], dtype=object))

In [62]:
# Thus far, we’ve only factorized lists (which are internally coerced to NumPy arrays). 
# When factorizing pandas objects, the type of uniques will differ. 
# For Categoricals, a Categorical is returned.
cat = pd.Categorical(['b', 'a', 'a', 'c', 'd'], categories=['a', 'b', 'c'])
codes, uniques = pd.factorize(cat)
codes, uniques

(array([ 0,  1,  1,  2, -1]), [b, a, c]
 Categories (3, object): [a, b, c])

8. `pandas.unique` Hash table-based unique. Uniques are returned in order of appearance. This does NOT sort.

In [63]:
pd.unique(pd.Series([2, 1, 3, 3]))

array([2, 1, 3])

In [64]:
pd.unique(pd.Series([2] + [1] * 5))

array([2, 1])

In [65]:
pd.unique(pd.Series([pd.Timestamp('20160101', tz='US/Eastern'),
                     pd.Timestamp('20160101', tz='US/Eastern')]))

<DatetimeArray>
['2016-01-01 00:00:00-05:00']
Length: 1, dtype: datetime64[ns, US/Eastern]

In [66]:
pd.unique(pd.Index([pd.Timestamp('20160101', tz='US/Eastern'),
                    pd.Timestamp('20160101', tz='US/Eastern')]))

DatetimeIndex(['2016-01-01 00:00:00-05:00'], dtype='datetime64[ns, US/Eastern]', freq=None)

In [67]:
pd.unique(list('baabc'))

array(['b', 'a', 'c'], dtype=object)

In [68]:
pd.unique(pd.Series(pd.Categorical(list('baabc'),
                                   categories=list('abc'),
                                   ordered=True)))

[b, a, c]
Categories (3, object): [a < b < c]

9. `pandas.wide_to_long` Wide panel to long format. Less flexible but more user-friendly than melt.

In [69]:
np.random.seed(123)
df = pd.DataFrame({"A1970" : {0 : "a", 1 : "b", 2 : "c"},
                   "A1980" : {0 : "d", 1 : "e", 2 : "f"},
                   "B1970" : {0 : 2.5, 1 : 1.2, 2 : .7},
                   "B1980" : {0 : 3.2, 1 : 1.3, 2 : .1},
                   "X"     : dict(zip(range(3), np.random.randn(3)))
                  })
df["id"] = df.index
df

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,-1.085631,0
1,b,e,1.2,1.3,0.997345,1
2,c,f,0.7,0.1,0.282978,2


In [70]:
pd.wide_to_long(df, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-1.085631,a,2.5
1,1970,0.997345,b,1.2
2,1970,0.282978,c,0.7
0,1980,-1.085631,d,3.2
1,1980,0.997345,e,1.3
2,1980,0.282978,f,0.1


### 2. Top-level missing data
* pandas.isna, pandas.isnull, pandas.notna, pandas.notnull

`pandas.isna` and `pandas.null` Detect missing values for an array-like object. This function takes a scalar or array-like object and indicates whether values are missing (NaN in numeric arrays, None or NaN in object arrays, NaT in datetimelike).

In [71]:
pd.isna('dog'), pd.isna(np.nan)

(False, True)

In [72]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
pd.isna(array)

array([[False,  True, False],
       [False, False,  True]])

In [73]:
index = pd.DatetimeIndex(["2017-07-05", "2017-07-06", None,
                          "2017-07-08"])
pd.isna(index)

array([False, False,  True, False])

In [74]:
df = pd.DataFrame([['ant', 'bee', 'cat'], ['dog', None, 'fly']])
pd.isna(df)

,0,1,2
0,False,False,False
1,False,True,False


In [75]:
pd.isna(df[1])

0    False
1     True
Name: 1, dtype: bool

In [76]:
pd.isnull(df)

,0,1,2
0,False,False,False
1,False,True,False


In [77]:
pd.notna(df)

,0,1,2
0,True,True,True
1,True,False,True


In [78]:
pd.notnull(df)

,0,1,2
0,True,True,True
1,True,False,True


### 3. Top-level conversion

* pandas.to_numeric

`pandas.to_numeric` Convert argument to a numeric type.

In [79]:
s = pd.Series(['1.0', '2', -3])
pd.to_numeric(s)

0    1.0
1    2.0
2   -3.0
dtype: float64

In [80]:
pd.to_numeric(s, downcast='float')

0    1.0
1    2.0
2   -3.0
dtype: float32

In [81]:
pd.to_numeric(s, downcast='signed')

0    1
1    2
2   -3
dtype: int8

In [82]:
s = pd.Series(['apple', '1.0', '2', -3])
pd.to_numeric(s, errors='ignore')

0    apple
1      1.0
2        2
3       -3
dtype: object

In [83]:
pd.to_numeric(s, errors='coerce')

0    NaN
1    1.0
2    2.0
3   -3.0
dtype: float64

### 4. Top-level dealing with datetimelike

* pandas.to_datetime, pandas.to_timedelta
* pandas.date_range, pandas.bdate_range, pandas.period_range, pandas.timedelta_range
* pandas.infer_freq

1. `pandas.to_datetime` Convert argument to datetime.

In [84]:
# pd.to_datetime?

In [85]:
#Assembling a datetime from multiple columns of a DataFrame. 
# The keys can be common abbreviations like [‘year’, ‘month’, ‘day’, ‘minute’, ‘second’, ‘ms’, ‘us’, ‘ns’]) or plurals of the same
df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5]})
pd.to_datetime(df)

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]

In [86]:
pd.to_datetime('19000101', format='%Y%m%d', errors='ignore')

Timestamp('1900-01-01 00:00:00')

In [87]:
pd.to_datetime('12/10/11', dayfirst=True)

Timestamp('2011-10-12 00:00:00')

`pandas.to_timedelta` Convert argument to timedelta.

In [88]:
pd.to_timedelta('15.5us')

Timedelta('0 days 00:00:00.000015')

### 5. Top-level dealing with intervals

* pandas.interval_range

### 6. Top-level evaluation

* pandas.eval

### 7. Hashing

* pandas.util.has_array
* pandas.util.has_pandas_objects

### 8. Testing

* pandas.test